In [ ]:
robyn_api_key=

In [8]:
from openai import OpenAI

client = OpenAI(api_key=robyn_api_key)

response = client.models.list()
for model in response.data:
    print(model.id)


gpt-4o-audio-preview-2024-12-17
dall-e-3
dall-e-2
gpt-4o-audio-preview-2024-10-01
gpt-4o-realtime-preview-2024-10-01
gpt-4o-transcribe
gpt-4o-mini-transcribe
gpt-4o-realtime-preview
babbage-002
tts-1-hd-1106
text-embedding-3-large
gpt-4
text-embedding-ada-002
tts-1-hd
gpt-4o-mini-audio-preview
gpt-4-0125-preview
gpt-4o-audio-preview
gpt-4-turbo-preview
o1-preview-2024-09-12
gpt-4o-mini-realtime-preview
gpt-4o-mini-realtime-preview-2024-12-17
gpt-3.5-turbo-instruct-0914
gpt-4o-mini-search-preview
tts-1-1106
davinci-002
gpt-3.5-turbo-1106
gpt-4-turbo
gpt-4o-realtime-preview-2024-12-17
gpt-3.5-turbo-instruct
gpt-3.5-turbo
chatgpt-4o-latest
gpt-4o-mini-search-preview-2025-03-11
gpt-4o-2024-11-20
whisper-1
gpt-4o-2024-05-13
gpt-3.5-turbo-16k
gpt-4-turbo-2024-04-09
gpt-4-1106-preview
o1-preview
gpt-4-0613
gpt-4o-search-preview
gpt-4.5-preview
gpt-4.5-preview-2025-02-27
gpt-4o-search-preview-2025-03-11
tts-1
omni-moderation-2024-09-26
o1
o1-2024-12-17
text-embedding-3-small
gpt-4o
o3-mini-202

In [ ]:
import pandas as pd
import time
import re
import os
from tqdm import tqdm
from openai import OpenAI

client = OpenAI(api_key=robyn_api_key)

# === File location ===
input_path = "2.1 advisor_reports_summary.xlsx"
output_path = "3.1 etf_with_ark_analysis_clean.xlsx"

# === Loading raw data ===
df = pd.read_excel(input_path)

# === Loading existing results or start new ===
if os.path.exists(output_path):
    df_out = pd.read_excel(output_path)
    print("✅ Loaded existing results, continuing from last save...")
else:
    df_out = df.copy()
    df_out["Investment Strategy"] = ""
    df_out["Comparable to ARK ETF?"] = ""
    df_out["Similar to ARK ETF"] = ""
    df_out["Reason"] = ""
    print("🆕 Starting fresh analysis...")

# === Set the model ===
gpt_model = "gpt-4o"
batch_size = 10
total_rows = df_out.shape[0]

# === How many remaining indexes ===
remaining_indices = [i for i in range(total_rows) if pd.isna(df_out.loc[i, "Investment Strategy"]) or df_out.loc[i, "Investment Strategy"] == ""]

# === Main process ===
for idx, i in enumerate(tqdm(remaining_indices, desc="Processing ETFs", unit="etf")):
    ticker = df_out.loc[i, "Ticker"]
    name = df_out.loc[i, "Name"]
    description = df_out.loc[i, "Extracted Text"]

    prompt = f"""You are a professional ETF analyst.

        Given the following ETF's ticker, name, and description, your task is as follows:

        1. Generate a clear and accurate summary of the ETF’s investment strategy.
        - Use the provided description and your own knowledge.
        - If the description is vague or lacks sufficient detail, say "Unknown" and do not make up content.

        2. Then, determine whether this ETF is comparable to any of the following ARK Invest ETFs based on thematic overlap and investment focus:

        - ARKK: disruptive innovation across sectors (e.g. AI, robotics, genomics)
        - ARKG: genomics, gene editing, molecular biology, precision medicine
        - ARKW: next-gen internet, AI, cloud, streaming, blockchain, digital content
        - ARKF: fintech, digital payments, blockchain finance, neobanks
        - ARKQ: robotics, EVs, drones, 3D printing, autonomous systems
        - ARKX: space exploration, satellites, aerospace defense
        - ARKB: bitcoin, digital assets, cryptocurrency
        - IZRL: Israeli tech and innovation
        - PRNT: 3D printing and additive manufacturing

        3. In your response, use this structure:

        Investment Strategy: <concise summary or 'Unknown'>

        Comparable to ARK ETF?: <Yes or No>

        If Yes, Similar to: <ARK ETF ticker>

        Reason: <1-3 sentence explanation based on thematic or strategic alignment>

        ---

        Ticker: {ticker}
        Name: {name}
        Description: {description}
        """

    try:
        response = client.chat.completions.create(
            model=gpt_model,
            messages=[
                {"role": "system", "content": "You are a professional ETF analyst."},
                {"role": "user", "content": prompt}
            ],
            temperature=0.4
        )

        content = response.choices[0].message.content.strip()

        strategy = re.search(r"Investment Strategy:\s*(.*)", content, re.IGNORECASE)
        comparable = re.search(r"Comparable to ARK ETF\?:\s*(Yes|No)", content, re.IGNORECASE)
        similar = re.search(r"Similar to:\s*([A-Z]{4})", content)
        reason = re.search(r"Reason:\s*(.*)", content, re.IGNORECASE)

        df_out.loc[i, "Investment Strategy"] = strategy.group(1).strip() if strategy else "Parse Error"
        df_out.loc[i, "Comparable to ARK ETF?"] = comparable.group(1).strip() if comparable else "Parse Error"
        df_out.loc[i, "Similar to ARK ETF"] = similar.group(1).strip() if similar else ""
        df_out.loc[i, "Reason"] = reason.group(1).strip() if reason else "Parse Error"

    except Exception as e:
        df_out.loc[i, "Investment Strategy"] = "Error"
        df_out.loc[i, "Comparable to ARK ETF?"] = "Error"
        df_out.loc[i, "Similar to ARK ETF"] = ""
        df_out.loc[i, "Reason"] = f"Error: {str(e)}"

    # Auto save
    if idx % batch_size == batch_size - 1 or idx == len(remaining_indices) - 1:
        df_out.to_excel(output_path, index=False)
        print(f"💾 Saved progress at row {i + 1} → {output_path}")

    time.sleep(1.2)


🆕 Starting fresh analysis...


Processing ETFs:   1%|          | 9/1468 [00:26<1:09:59,  2.88s/etf]

💾 Saved progress at row 10 → etf_with_ark_analysis_clean.xlsx


Processing ETFs:   1%|▏         | 19/1468 [00:56<1:09:54,  2.89s/etf]

💾 Saved progress at row 20 → etf_with_ark_analysis_clean.xlsx


...

Processing ETFs:  99%|█████████▉| 1459/1468 [1:16:00<00:26,  2.92s/etf]

💾 Saved progress at row 1460 → etf_with_ark_analysis_clean.xlsx


Processing ETFs: 100%|█████████▉| 1467/1468 [1:16:23<00:03,  3.04s/etf]

💾 Saved progress at row 1468 → etf_with_ark_analysis_clean.xlsx


Processing ETFs: 100%|██████████| 1468/1468 [1:16:26<00:00,  3.12s/etf]
